In [1]:
import glob
from jira import JIRA
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
VECTORSTORE_PATH = "../../vs_jira"

PROMPT_GEN_US = """
    Vous êtes un analyste d'affaires expert spécialisé dans la rédaction de récits utilisateur de haute qualité, optimisés pour les tests et l'assurance qualité. Vous comprenez que des récits utilisateur bien rédigés facilitent la création de cas de test exhaustifs et réduisent les ambiguïtés lors du développement.

## Principes fondamentaux

### Structure obligatoire
Chaque récit utilisateur DOIT suivre le format :
- **En tant que** [rôle/persona spécifique]
- **Je veux** [action/fonctionnalité claire et atomique]
- **Afin de** [bénéfice mesurable ou valeur apportée]

### Critères d'acceptation testables
Chaque récit DOIT inclure des critères d'acceptation qui sont :
- **Spécifiques** : Pas d'ambiguïté sur ce qui doit être vérifié
- **Mesurables** : Résultats quantifiables ou observables
- **Vérifiables** : Peuvent être traduits directement en cas de test
- **Complets** : Couvrent le chemin nominal ET les cas d'erreur

### Format des critères d'acceptation
Utilisez le format Given/When/Then (Étant donné/Quand/Alors) :
- **Étant donné** [contexte/précondition]
- **Quand** [action de l'utilisateur]
- **Alors** [résultat attendu observable]

## Exigences pour la testabilité

### Cas à couvrir obligatoirement
1. **Chemin nominal** (happy path) : Le scénario où tout fonctionne correctement
2. **Cas d'erreur** : Comportement en cas d'entrées invalides ou d'erreurs système
3. **Cas limites** : Valeurs aux frontières (min, max, vide, null)
4. **Cas aux limites** : Comportement à la limite exacte des plages valides

### Données de test
- Spécifiez les types de données attendus (format, longueur, caractères autorisés)
- Indiquez les valeurs limites explicitement
- Précisez les dépendances de données entre champs

### Messages et retours utilisateur
- Décrivez le texte exact des messages d'erreur attendus
- Spécifiez où et comment les messages doivent s'afficher
- Indiquez la durée d'affichage si applicable

## Règles de rédaction

### Atomicité
- Un récit = une seule fonctionnalité testable
- Si un récit nécessite plus de 8-10 critères d'acceptation, découpez-le

### Précision
- Évitez les termes vagues : "rapide", "convivial", "facile"
- Utilisez des valeurs concrètes : "en moins de 3 secondes", "en 2 clics maximum"
- Nommez les éléments d'interface avec précision

### Indépendance
- Chaque récit doit être testable indépendamment
- Précisez explicitement les dépendances si elles existent

## Structure de sortie

"""

In [3]:
JIRA_URL = os.environ.get("JIRA_URL")
EMAIL = os.environ.get("EMAIL")
API_TOKEN = os.environ.get("JIRA_API_TOKEN")
PROJECT_KEY = "MJ"

jira = JIRA(
    server=JIRA_URL,
    basic_auth=(EMAIL, API_TOKEN)
)

user = jira.current_user()
print(f"Connecté en tant que: {user}")

Connecté en tant que: 712020:4da4c4ca-c19f-47de-bbb6-3c4574de0abd


In [4]:
import getpass

if not os.environ.get("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

model = AzureChatOpenAI(
    azure_endpoint="https://menoua.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview",
    azure_deployment="gpt-41",
    api_version="2025-01-01-preview",
    temperature=0,
    use_responses_api=False,
)

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    chunk_size=64,
    # max_retries=10,
    # retry_min_seconds=10,
    # retry_max_seconds=60,
)

# Extraction JIRA

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import tool
from typing import Literal

def get_all_issues(project_key: str) -> list:
    jql = f"project = {project_key} ORDER BY created ASC"
    issues = jira.search_issues(
        jql,
        maxResults=False,
        fields="summary,issuetype,status,parent,description,created,updated"
    )
    return issues

def issues_to_documents(issues) -> list[Document]:
    documents = []

    for issue in issues:
        fields = issue.fields

        content_parts = [
            f"Clef: {issue.key}",
            f"Type: {fields.issuetype.name}",
            f"Titre: {fields.summary}",
            f"Statut: {fields.status.name}",
        ]

        if hasattr(fields, "parent") and fields.parent:
            content_parts.append(f"Epic parent: {fields.parent.key}")

        if fields.description:
            content_parts.append(f"Description: {fields.description}")

        page_content = "\n".join(content_parts)

        metadata = {
            "key": issue.key,
            "type": fields.issuetype.name,
            "status": fields.status.name,
            "summary": fields.summary,
            "parent": fields.parent.key if hasattr(fields, "parent") and fields.parent else None,
            "created": fields.created,
            "url": f"{JIRA_URL}/browse/{issue.key}"
        }

        documents.append(Document(page_content=page_content, metadata=metadata))

    return documents

# Décomposition

In [6]:
def split_documents(documents: list[Document], chunk_size: int = 500, chunk_overlap: int = 100) -> list[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    all_splits = text_splitter.split_documents(documents)
    print(f"Répartition en {len(all_splits)} sous-documents.")
    return all_splits

# BDDv

In [7]:
def create_or_load_vectorstore(documents: list[Document], path: str, force_recreate: bool = False) -> FAISS:

    if not force_recreate and os.path.exists(path):
        print(f"Chargement de la BDD existante depuis '{path}'...")
        vectorstore = FAISS.load_local(
            path,
            embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"BDD chargée avec {vectorstore.index.ntotal} vecteurs.")
    else:
        print("Création de la BDD FAISS...")
        vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)
        vectorstore.save_local(path)
        print(f"BDD créée et sauvegardée avec {vectorstore.index.ntotal} vecteurs.")

    return vectorstore

# Exécution

In [8]:
print("Récupération des tickets Jira...")
issues = get_all_issues(PROJECT_KEY)
print(f"   → {len(issues)} tickets récupérés")

print("Conversion en documents...")
documents = issues_to_documents(issues)
print(f"   => {len(documents)} documents créés")

print("Découpage en chunks...")
all_splits = split_documents(documents, chunk_size=500, chunk_overlap=100)

print("BDD FAISS...")
vectorstore = create_or_load_vectorstore(
    all_splits,
    VECTORSTORE_PATH,
    force_recreate=True  # False pour réutiliser l'existant
)

print("\nTest de recherche:")
test_query = "inscription tournoi"
results = vectorstore.similarity_search(test_query, k=2)
for i, doc in enumerate(results, 1):
    print(f"   [{i}] {doc.metadata.get('key')} - {doc.metadata.get('summary')}")

Récupération des tickets Jira...
   → 49 tickets récupérés
Conversion en documents...
   => 49 documents créés
Découpage en chunks...
Répartition en 111 sous-documents.
BDD FAISS...
Création de la BDD FAISS...
BDD créée et sauvegardée avec 111 vecteurs.

Test de recherche:
   [1] MJ-57 - Seamless and Personalized Reservation System
   [2] MJ-32 - Seamless and Personalized Reservation System


In [9]:
@tool(response_format="content_and_artifact")
def retrieve_context(
        query: str,
        section: Literal[
            "Amélioration de récit utilisateur",
            "Génération de récit utilisateur",
            "Génération de cas de test",
            "Autre"
        ]
):
    """
    Ajoute du contexte à la requête en recherchant des tickets Jira similaires.

    Args:
        query: La requête de l'utilisateur
        section: Le type d'opération demandée
    """
    retrieved_docs = vectorstore.similarity_search(query, k=3)
    serialized = "\n\n".join(
        f"Source: {doc.metadata}\nContent: {doc.page_content}"
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [10]:
from langchain.agents import create_agent

tools = [retrieve_context]

prompt = (
    PROMPT_GEN_US
)
agent = create_agent(model, tools, system_prompt=prompt)

query = (
    "Quels sont les epics dans Magic Jira?\n"
)

for event in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Quels sont les epics dans Magic Jira?

================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_l3yvMsXhq8MHN5fj1b9a8W9L)
 Call ID: call_l3yvMsXhq8MHN5fj1b9a8W9L
  Args:
    query: epics dans Magic Jira
    section: Autre
================================= Tool Message =================================
Name: retrieve_context

Source: {'key': 'MJ-43', 'type': 'Epic', 'status': 'À faire', 'summary': 'Scalability and Future-Proofing', 'parent': None, 'created': '2026-01-28T17:28:50.678+0100', 'url': 'https://menoua.atlassian.net/browse/MJ-43'}
Content: Clef: MJ-43
Type: Epic
Titre: Scalability and Future-Proofing
Statut: À faire

Source: {'key': 'MJ-53', 'type': 'Epic', 'status': 'À faire', 'summary': 'Immersive and Innovative User Experience', 'parent': None, 'created': '2026-01-28T17:45:44.208+0100', 'url': 'https://menoua.atlassian.net/browse/MJ-

## Appel unique

In [11]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """injecte le contexte"""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vectorstore.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
            PROMPT_GEN_US +
            f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [12]:
query = "Quels sont les epics dans Magic Jira?\n"
for step in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Quels sont les epics dans Magic Jira?

================================== Ai Message ==================================

Voici la liste des **epics** présents dans Magic Jira, selon les informations fournies :

---

### Clef: MJ-52
- **Type** : Epic
- **Titre** : Immersive and Innovative User Experience
- **Statut** : À faire

---

### Clef: MJ-53
- **Type** : Epic
- **Titre** : Immersive and Innovative User Experience
- **Statut** : À faire

---

### Clef: MJ-43
- **Type** : Epic
- **Titre** : Scalability and Future-Proofing
- **Statut** : À faire

---

### Clef: MJ-73
- **Type** : Epic
- **Titre** : Compatibility and Accessibility
- **Statut** : À faire

---

**Résumé :**
- MJ-52 : Immersive and Innovative User Experience
- MJ-53 : Immersive and Innovative User Experience
- MJ-43 : Scalability and Future-Proofing
- MJ-73 : Compatibility and Accessibility

> **Remarque** : Les epics MJ-52 et MJ-53 porten